In [ ]:
#! pip install imutils

Coding instructions courtesy of: 

Adrian Garrido

https://www.pyimagesearch.com/2018/09/10/keras-tutorial-how-to-get-started-with-keras-deep-learning-and-python/

In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras import backend as K
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

Using TensorFlow backend.


## Model Construction - CNN
### Construct a VGGNet 

In [2]:
# Instantiate CNN model
class SmallVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
 
        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
            
        # CONV => RELU => POOL layer set
        model.add(Conv2D(32, (3, 3), padding="same",
            input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
            
            
            
        # (CONV => RELU) * 2 => POOL layer set
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
            
        
        # (CONV => RELU) * 3 => POOL layer set
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
 
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
 
        # return the constructed network architecture
        return model

### Gather and Pre-Process Images

In [3]:
# initialize the data and labels
data = []
labels = []
 
# grab the image paths and randomly shuffle them
shiba_path = sorted(list(paths.list_images('../downloads/shiba')))
fox_path = sorted(list(paths.list_images('../downloads/fox')))
combine_path = [shiba_path,fox_path]
random.seed(42)

# loop over the input data dictionaries
for path in combine_path:
    path = random.shuffle(path)

# loop over the input images
for imagePaths in combine_path:
    for imagePath in imagePaths:
        try:
            # load the image, resize the image to be 32x32 pixels (ignoring
            # aspect ratio), flatten the image into 32x32x3=3072 pixel image
            # into a list, and store the image in the data list
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (64, 64))
            data.append(image)

            # extract the class label from the image path and update the
            # labels list
            label = imagePath.split(os.path.sep)[-2].split('/')[-1]
            labels.append(label)
        except:
            pass

In [ ]:
len(data)

In [ ]:
# Check Label to ensure proper processing
labels[780:790]

In [ ]:
# Check Label to ensure proper processing
labels[1879:1889]

In [4]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
data.shape

In [ ]:
labels.shape

In [ ]:
# Check labels for proper data type
labels

### Run Model

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.25, random_state=42)
 
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
data.shape

In [ ]:
trainY.shape

In [ ]:
trainX.shape

In [ ]:
len(trainX)

In [ ]:
len(testX)

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")
 
# initialize our VGG-like Convolutional Neural Network
model = SmallVGGNet.build(width=64, height=64, depth=3,
    classes=len(lb.classes_))

In [ ]:
model

In [ ]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 20
BS = 32
 
# initialize the model and optimizer (you'll want to use
# categorical_crossentropy for 3-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
 
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

In [ ]:
trainX.shape

In [ ]:
# Save the model
model.save('model_CNN_1')

### Evaluate Model

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1), target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (VGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
#plt.savefig(args["plot"])
 
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save('model_3B')
# f = open(args["label_bin"], "wb")
# f.write(pickle.dumps(lb))
# f.close()

In [ ]:
H.history['val_acc'][-1]

In [ ]:
# Show prediction results
predictions

In [ ]:
# Set index to the largest number per row
# Set variable equal to the prediction results for future analysis
model_predictions = predictions.argmax(axis = 1)

In [ ]:
# Set index to the largest number per row
# Set variable equal to the prediction results for future analysis
testY = testY.argmax(axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(testY, model_predictions)

## Actual vs Prediction Analysis

In [ ]:
import pandas as pd

In [ ]:
# Set array into dataframe for further analysis
df_predictions = pd.DataFrame(model_predictions)
df_predictions.columns = ['preds']
df_predictions.head()

In [ ]:
# Set array into dataframe for further analysis
df_actuals = pd.DataFrame(testY)
df_actuals.columns = ['actuals']
df_actuals.head()

In [ ]:
# Create a dataframe that includes both actuals and prediction
actuals_pred_df = pd.concat([df_actuals, df_predictions], axis = 1, join= 'outer')

In [ ]:
# Check dataframe 
actuals_pred_df.head()

In [ ]:
# Create a function that will output a dataframe where the actuals and predictions did not match

def pred_errors(df, col1, col2):
    errors = []
    for i in range(len(df)):
        if df[col1][i] != df[col2][i]:
            errors.append(i)
    return df.loc[errors]

# running function on the previously created dataframe for actuals and predictions
    
pred_errors = pred_errors(actuals_pred_df, 'actuals', 'preds')

In [ ]:
# Show total numbers of incorrect predictions
pred_errors.count()

In [ ]:
# Desctibe the prediction errors
pred_errors.describe()

In [ ]:
# Display a histogram of the prediction errors
pred_errors.hist();

## Make Predictions  - Out of Sample

In [ ]:
from keras.models import load_model
from keras.preprocessing import image

In [ ]:
# Create input images
image_1 = cv2.imread('../train/test/test (1).jpg')
image_2 = cv2.imread('../train/test/test (2).jpg')
image_3 = cv2.imread('../train/test/test (3).jpg')

In [ ]:
# Display image
plt.imshow(image_1);

In [ ]:
# Display image
plt.imshow(image_2);

In [ ]:
# Display image
plt.imshow(image_3);

In [ ]:
# load the input images and resize it to the target spatial dimensions
#image = image.convert('RGB')
image_1 = cv2.resize(image_1, (64,64))
image_2 = cv2.resize(image_2, (64,64))
image_3 = cv2.resize(image_3, (64,64))
 
# scale the pixel values to [0, 1]
image_1 = image_1.astype("float") / 255.0
image_2 = image_2.astype("float") / 255.0
image_3 = image_3.astype("float") / 255.0

In [ ]:
# Make predictions for out of sample testing images
image_pred_1 = np.expand_dims(image_1, axis = 0)
make_pred_1 = model.predict(image_pred_1)

image_pred_2 = np.expand_dims(image_2, axis = 0)
make_pred_2 = model.predict(image_pred_2)

image_pred_3 = np.expand_dims(image_3, axis = 0)
make_pred_3 = model.predict(image_pred_3)

In [ ]:
make_pred_1

In [ ]:
make_pred_2

In [ ]:
make_pred_3

## LIVE DEMO

In [ ]:
# Create input images
image_1 = cv2.imread('../train/test/Westminster_Abbey.jpg')
image_2 = cv2.imread('../train/test/sensoji_temple.jpg')
image_3 = cv2.imread('../train/test/Big_Buddha.jpg')

In [ ]:
# Display image
plt.imshow(image_1);

In [ ]:
# Display image
plt.imshow(image_2);

In [ ]:
# Display image
plt.imshow(image_3);

In [ ]:
# load the input images and resize it to the target spatial dimensions
#image = image.convert('RGB')
image_1 = cv2.resize(image_1, (64,64))
image_2 = cv2.resize(image_2, (64,64))
image_3 = cv2.resize(image_3, (64,64))
 
# scale the pixel values to [0, 1]
image_1 = image_1.astype("float") / 255.0
image_2 = image_2.astype("float") / 255.0
image_3 = image_3.astype("float") / 255.0

In [ ]:
# Make predictions for out of sample testing images
image_pred_1 = np.expand_dims(image_1, axis = 0)
make_pred_1 = model.predict(image_pred_1)

image_pred_2 = np.expand_dims(image_2, axis = 0)
make_pred_2 = model.predict(image_pred_2)

image_pred_3 = np.expand_dims(image_3, axis = 0)
make_pred_3 = model.predict(image_pred_3)

In [ ]:
# Display prediction
make_pred_1

In [ ]:
# Display prediction
make_pred_2

In [ ]:
# Display prediction
make_pred_3